In [1]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
from vectorbtpro import *
from time import time
import helpers as pth
import platform
from dotenv import load_dotenv
import scipy.stats as stats
import time
import helpers as pth
from numba import njit
import talib

theme = "light"
vbt.settings.set_theme(theme)

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12, 7)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]
plt.style.use("classic" if theme == "light" else "dark_background")

if platform.system().lower() == "windows":
    base_data_path = "H:\\phitech-data\\01_raw"
else:
    from core_chains.simple.llm import make_Q_chain

    base_data_path = "../../phitech-data/01_raw"
    load_dotenv("../../sandatasci-core/credentials")
    Q = make_Q_chain("gpt-4o-instance1", __vsc_ipynb_file__)

/Users/e1211913/miniconda3/envs/trading/lib/python3.12/site-packages/telegram/utils/request.py:49: UserWarning: python-telegram-bot is using upstream urllib3. This is allowed but not supported by python-telegram-bot maintainers.
  warnings.warn(


logging mode: dev


In [2]:
%%html
<style>
.dataframe {
    font-size: 9pt; /* Adjust font size as needed */
}
</style>

In [12]:
start = "2024-10-01"
end = "2024-10-10"
symbols = ["MES"]
chunk_size = 2

chunks = pth.get_date_chunks(start, end, chunk_size=1)
chunks

[('2024-10-01', '2024-10-02'),
 ('2024-10-02', '2024-10-03'),
 ('2024-10-03', '2024-10-04'),
 ('2024-10-04', '2024-10-05'),
 ('2024-10-05', '2024-10-06'),
 ('2024-10-06', '2024-10-07'),
 ('2024-10-07', '2024-10-08'),
 ('2024-10-08', '2024-10-09'),
 ('2024-10-09', '2024-10-10')]

In [53]:
chunk_start, chunk_end = chunks[0]

df = pth.SierraChartData.ffill(
    pth.SierraChartData.pull(symbols, start=chunk_start, end=chunk_end)
)
df = df.resample("1min")

In [70]:
period = 10
indicator = vbt.talib("ATR").run(df.get("High"), df.get("Low"), df.get("Close"), period)

In [81]:
multiplier = 2
fig = df.get("Close").vbt.plot(height=480)
(df.get("Close") + multiplier * indicator.atr).vbt.plot(fig=fig)
(df.get("Close") - multiplier * indicator.atr).vbt.plot(fig=fig)

FigureWidget({
    'data': [{'name': 'MES',
              'showlegend': True,
              'type': 'scatter',
              'uid': '26a63e2e-7522-4fb4-82c9-77ae60f931e0',
              'x': array([datetime.datetime(2024, 10, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 10, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([5929.  , 5928.  , 5927.5 , ..., 5873.  , 5871.  , 5871.25])},
             {'name': "(10, 'MES')",
              'showlegend': True,
              'type': 'scatter',
              'uid':

In [82]:
# TODO: continue here with building the pipeline when you're back at this from home.